# Example: differential network analysis

## Preamble
Complementary sketching implemented in compsket.py

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn import linear_model
import sklearn
import time
from compsket import *

## Generate data matrix X (can be skipped if csv file is already available)

In [2]:
from os.path import exists
if not exists('CD4_TREG_in_thymus.csv.gz'):
    import scanpy as sc
    adata = sc.read('NKT_thymus_panfetal.h5ad')
    adata = adata[adata.obs['anno_lvl_2_final_clean'].isin(['CD4+T','TREG'])]
    adata = adata[adata.obs['Sort_id'].isin(['CD45N', 'CD45P', 'TOT'])]
    
    X = adata.X.todense()
    gene_var = np.array(np.var(X, axis=0, ddof=1)).flatten()
    filter = gene_var >= 1
    X = X[:, filter]
    
    gene_names = np.array(adata.var_names[filter])
    cell_id = np.array(adata.obs_names)
    cell_types = np.array(adata.obs['anno_lvl_2_final_clean'])
    CD4_filter = cell_types == 'CD4+T'
    
    df = pd.DataFrame(X, index=cell_id, columns=gene_names)
    df = pd.concat([adata.obs['anno_lvl_2_final_clean'], df], axis=1)
    df.to_csv('CD4_TREG_in_thymus.csv', index=True, header=True, sep=',')

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


## Read data matrix from file directly

In [2]:
dat = pd.read_csv('CD4_TREG_in_thymus.csv.gz', index_col=0)
X = np.array(dat.iloc[:,1:])
gene_names = np.array(dat.columns[1:])
CD4_filter = dat['anno_lvl_2_final_clean'] == 'CD4+T'
X.shape

(7816, 4123)

## Perform differential network analysis

In [7]:
X1 = np.array(X[CD4_filter, :])
X2 = np.array(X[~CD4_filter, :])
"X1 shape = {}, X2 shape = {}".format(X1.shape, X2.shape)

'X1 shape = (4852, 4123), X2 shape = (2964, 4123)'

In [4]:
t = time.time()
# for a quicker test run, may set nodes = [0, 133, 180] as a parameter of differentialNetworkAnalysis()
# change nodes = None to run for all nodes
result = differentialNetworkAnalysis(X1, X2, nodes = [0, 133, 180], num_partners=8, trace=True)
print('{} seconds elapsed.'.format(time.time() - t))

Computing complementary sketches...
Computing 3/3 node...
Finished: 2 significant node found.
55.70351457595825 seconds elapsed.


## tidy up results using gene names instead of indices

In [8]:
result = result.loc[result['test_result'] == 1, :]
result.index = np.array(gene_names)[result.index]
result['interacting_partners'] = [gene_names[v] for v in result['interacting_partners']]

In [9]:
result

,test_stat,test_result,interacting_partners
IKZF2,72.240301,1,"[MT-ND4L, HLA-B, MT-ATP8, ETS1, FYB1, JUNB, RN..."
FOXP3,297.40523,1,"[MT-ND4L, MT-ATP8, S100A4, CD96, ISG20, BIRC2,..."
